<a href="https://colab.research.google.com/github/anasmghouri/Coursera_Capstone/blob/main/The_Battle_of_Neighborhoods_p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
import re

!pip install geopy
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim

In [40]:
url = "https://de.wikipedia.org/wiki/Stadtbezirke_M%C3%BCnchens"
html = requests.get(url).content
df_list = pd.read_html(html)
Mnc_popp = df_list[0]
Mnc_popp.rename(columns={'Nr.': 'No.', 'Stadtbezirk': 'Borough', 'Fläche(km²)': 'Area(km²)', 'Einwohner': 'Population', 'Dichte(Einw./km²)': 'Density(Pop/km²)', 'Ausländer(%)': 'Foreigners(%)'}, inplace=True)
Mnc_popp['Area(km²)'] = Mnc_popp['Area(km²)'].div(100).round(2)
Mnc_popp['Foreigners(%)'] = Mnc_popp['Foreigners(%)'].div(10).round(2)
Mnc_popp.head()

,No.,Borough,Area(km²),Population,Density(Pop/km²),Foreigners(%)
0,1.0,Altstadt-Lehel,3.15,21.126,6.716,26.0
1,2.0,Ludwigsvorstadt-Isarvorstadt,4.40,51.933,11.799,28.3
2,3.0,Maxvorstadt,4.30,51.834,12.060,25.6
3,4.0,Schwabing-West,4.36,68.935,15.800,22.8
4,5.0,Au-Haidhausen,4.22,61.654,14.611,23.5


In [41]:
Mnc_boroughs = df_list[1]
Mnc_boroughs.rename(columns={'Stadt-bezirks-nr.': 'Borough No.', 'Stadtbezirk': 'Borough', 'Stadtbezirksteile (Nr.)': 'Districts'}, inplace=True)
Mnc_boroughs['Districts'] = Mnc_boroughs['Districts'].str.replace(r'\(\d+\)', '')
Mnc_boroughs.head()

,Borough No.,Borough,Districts
0,1,Altstadt-Lehel,"Graggenau , Angerviertel , Hackenviertel , Kre..."
1,2,Ludwigsvorstadt-Isarvorstadt,"Gärtnerplatz , Deutsches Museum , Glockenbach ..."
2,3,Maxvorstadt,"Königsplatz , Augustenstraße , St. Benno , Mar..."
3,4,Schwabing-West,"Neuschwabing , Am Luitpoldpark , Schwere-Reite..."
4,5,Au-Haidhausen,"Maximilianeum , Steinhausen , Haidhausen-Nord ..."


In [42]:
# postal code data from: https://www.suche-postleitzahl.org/plz-gebiet/80
Mnc_postal_codes = pd.read_excel(r"https://github.com/anasmghouri/Coursera_Capstone/blob/main/MNC.xlsx?raw=true", sheet_name='PLZ')
Mnc_postal_codes.rename(columns={'PLZ': 'Postal Code', 'Stadtteil': 'Borough'}, inplace=True)
Mnc_postal_codes.head()

,Postal Code,Borough
0,80331,Altstadt-Lehel
1,80331,Ludwigsvorstadt-Isarvorstadt
2,80333,Altstadt-Lehel
3,80333,Maxvorstadt
4,80335,Altstadt-Lehel


In [43]:
# prices data 2020 from: https://wirkaufen24.de/immobilienpreise-munchen/
Mnc_prices = pd.read_excel(r"https://github.com/anasmghouri/Coursera_Capstone/blob/main/MNC.xlsx?raw=true", sheet_name='Price')
del Mnc_prices['Stadt']
Mnc_prices.rename(columns={'Postleitzahl': 'Postal Code', 'Straße': 'Street', 'm² im Ø': 'Objectm²', 'Ø - Preis': 'Object€', 'm² Ø in €': '€/m²'}, inplace=True)
Mnc_prices.head()

,Postal Code,Street,Objectm²,Object€,€/m²
0,80333,Augustenstraße,170.0,2195000,12911.0
1,80333,Barer Straße,66.0,1338000,20272.0
2,80333,Karlstr.,94.0,2600000,27659.0
3,80333,Maximiliansplatz,166.0,22000000,132530.0
4,80333,Richard-Wagner-Str.,211.0,4400000,20853.0


In [44]:
Mnc_bor_prices = pd.merge(Mnc_postal_codes,  
                      Mnc_prices,  
                      on ='Postal Code',  
                      how ='inner')

Mnc_bor_prices.head()

,Postal Code,Borough,Street,Objectm²,Object€,€/m²
0,80333,Altstadt-Lehel,Augustenstraße,170.0,2195000,12911.0
1,80333,Altstadt-Lehel,Barer Straße,66.0,1338000,20272.0
2,80333,Altstadt-Lehel,Karlstr.,94.0,2600000,27659.0
3,80333,Altstadt-Lehel,Maximiliansplatz,166.0,22000000,132530.0
4,80333,Altstadt-Lehel,Richard-Wagner-Str.,211.0,4400000,20853.0


**Fetching the Longitudes and Latitudes of the streets:**

In [45]:
Mnc_bor_prices['Latitude'] = None
Mnc_bor_prices['Longitude'] = None

In [46]:
URL = "https://geocode.search.hereapi.com/v1/geocode"
api_key = "kWhBD4nPku3EJIOCJaC2_lJlR7DcVqSoNEYYRksJDno"

for index, row in Mnc_bor_prices.iterrows():
  location = input(str(row['Postal Code']) + ' Munich')
  PARAMS = {'apikey':api_key,'q':location}
  r = requests.get(url = URL, params = PARAMS)
  data = r.json()
  row['Latitude'] = data['items'][0]['position']['lat']
  row['Longitude'] = data['items'][0]['position']['lng']

KeyboardInterrupt: ignored

In [47]:
URL = "https://geocode.search.hereapi.com/v1/geocode"
api_key = "kWhBD4nPku3EJIOCJaC2_lJlR7DcVqSoNEYYRksJDno"

location = input("Offenbachstraße 5, 81245 München") #taking user input
PARAMS = {'apikey':api_key,'q':location} 

# sending get request and saving the response as response object 
r = requests.get(url = URL, params = PARAMS) 
data = r.json()

latitude = data['items'][0]['position']['lat']
longitude = data['items'][0]['position']['lng']

KeyboardInterrupt: ignored

In [ ]:
latitude